In [1]:
# -*- coding: utf-8 -*-
# author：Cookly
from util import DataLoader, Features
from SBBTree_ONLINE import SBBTree
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import lightgbm as lgb

In [2]:
# test code
Data = DataLoader(
				FILE_jdata_sku_basic_info='../data/jdata_sku_basic_info.csv',
				FILE_jdata_user_action='../data/jdata_user_action.csv',
				FILE_jdata_user_basic_info='../data/jdata_user_basic_info.csv',
				FILE_jdata_user_comment_score='../data/jdata_user_comment_score.csv',
				FILE_jdata_user_order='../data/jdata_user_order.csv'
			)
# train data
TrainFeatures = Features(
						DataLoader=Data,
						PredMonthBegin = datetime(2017, 4, 1),
						PredMonthEnd = datetime(2017, 4, 30),
						FeatureMonthList = [(datetime(2017, 3, 1), datetime(2017, 3, 31), 1),\
									(datetime(2017, 1, 1), datetime(2017, 3, 31), 3),\
									(datetime(2016, 10, 1), datetime(2017, 3, 31), 6)],
						MakeLabel = True
					)
# pred data
PredFeatures = Features(
					DataLoader=Data,
					PredMonthBegin = datetime(2017, 5, 1),
					PredMonthEnd = datetime(2017, 5, 31),
					FeatureMonthList = [(datetime(2017, 4, 1), datetime(2017, 4, 30), 1),\
									(datetime(2017, 2, 1), datetime(2017, 4, 30), 3),\
									(datetime(2016, 11, 1), datetime(2017, 4, 30), 6)],
					MakeLabel = False
				)

In [3]:
train_features = TrainFeatures.TrainColumns
cols = TrainFeatures.IDColumns + TrainFeatures.LabelColumns + train_features
							
###########
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
###############################################################
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)

In [4]:
# train 下个月购买次数预测 回归模型
train_features = TrainFeatures.TrainColumns
train_label_BuyNum = 'Label_30_101_BuyNum'

X = TrainFeatures.data_BuyOrNot_FirstTime[train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum].values

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum] = model.predict(X_pred)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[1]	valid_0's l2: 0.756468	valid_0's auc: 0.802053
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 0.747011	valid_0's auc: 0.804084
[3]	valid_0's l2: 0.73781	valid_0's auc: 0.804226
[4]	valid_0's l2: 0.729222	valid_0's auc: 0.804758
[5]	valid_0's l2: 0.720966	valid_0's auc: 0.804752
[6]	valid_0's l2: 0.711872	valid_0's auc: 0.805354
[7]	valid_0's l2: 0.704883	valid_0's auc: 0.805688
[8]	valid_0's l2: 0.698406	valid_0's auc: 0.805775
[9]	valid_0's l2: 0.691333	valid_0's auc: 0.806619
[10]	valid_0's l2: 0.685772	valid_0's auc: 0.806738
[11]	valid_0's l2: 0.680924	valid_0's auc: 0.806742
[12]	valid_0's l2: 0.675552	valid_0's auc: 0.806872
[13]	valid_0's l2: 0.671319	valid_0's auc: 0.807277
[14]	valid_0's l2: 0.666608	valid_0's auc: 0.80735
[15]	valid_0's l2: 0.662788	valid_0's auc: 0.807539
[16]	valid_0's l2: 0.659222	valid_0's auc: 0.807669
[17]	valid_0's l2: 0.65521	valid_0's auc: 0.807838
[18]	valid_0's l2: 0.651972	valid_0's auc: 0.807963
[19]	valid_0'

In [5]:
###############################################################
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# train 当月首次购买时间预测 回归模型
train_label_FirstTime = 'Label_30_101_FirstTime'
X = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_label_FirstTime].values 

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values 

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime] = model.predict(X_pred)

[1]	valid_0's l2: 60.9599
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 60.7934
[3]	valid_0's l2: 60.6282
[4]	valid_0's l2: 60.4743
[5]	valid_0's l2: 60.3321
[6]	valid_0's l2: 60.1985
[7]	valid_0's l2: 60.0668
[8]	valid_0's l2: 59.9442
[9]	valid_0's l2: 59.8307
[10]	valid_0's l2: 59.7233
[11]	valid_0's l2: 59.6137
[12]	valid_0's l2: 59.5152
[13]	valid_0's l2: 59.4211
[14]	valid_0's l2: 59.3306
[15]	valid_0's l2: 59.2476
[16]	valid_0's l2: 59.1635
[17]	valid_0's l2: 59.0889
[18]	valid_0's l2: 59.0193
[19]	valid_0's l2: 58.9545
[20]	valid_0's l2: 58.8921
[21]	valid_0's l2: 58.8315
[22]	valid_0's l2: 58.7683
[23]	valid_0's l2: 58.7065
[24]	valid_0's l2: 58.6512
[25]	valid_0's l2: 58.5967
[26]	valid_0's l2: 58.5537
[27]	valid_0's l2: 58.5145
[28]	valid_0's l2: 58.472
[29]	valid_0's l2: 58.4365
[30]	valid_0's l2: 58.4025
[31]	valid_0's l2: 58.3562
[32]	valid_0's l2: 58.3204
[33]	valid_0's l2: 58.285
[34]	valid_0's l2: 58.2596
[35]	valid_0's l2: 58.2272
[36

In [8]:
# submit
columns = ['user_id'] + [train_label_BuyNum] + [train_label_FirstTime]
out_submit = PredFeatures.data_BuyOrNot_FirstTime[columns].sort_values(['Label_30_101_BuyNum'],ascending=False)
out_submit[train_label_FirstTime] = out_submit[train_label_FirstTime].map(lambda day: datetime(2017, 5, 1)+timedelta(days=int(day+0.49-1)))

out_submit = out_submit[['user_id']+[train_label_FirstTime]]
out_submit.columns = ['user_id','pred_date']
out_submit.head(50000).to_csv('../submit/predict-baseline-0.003.csv',index=False,header=True)